In [1]:
from _utils import *
from config import *

import time
import argparse
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import sparse
from scipy.sparse.linalg import spsolve as spsolve

import warnings
warnings.filterwarnings('ignore')

np.random.seed(28)

# 1 Functions

In [2]:
class args_config():
    def __init__(self, dataset_name):
        self.model_name = 'LATC_NN'
        self.model_result_save_path = os.path.join(result_folder_path, self.model_name)  
        create_folder(self.model_result_save_path)

        self.dataset_name = dataset_name
        self.dataset_result_save_path = os.path.join(self.model_result_save_path, self.dataset_name) 
        create_folder(self.dataset_result_save_path)
    
    def experiment_config(self, experiment_name):  
        self.experiment_result_save_path = os.path.join(self.dataset_result_save_path, experiment_name)
        create_folder(self.experiment_result_save_path)

        data_complete_path = os.path.join(self.experiment_result_save_path, 'data_complete')  
        best_data_complete_path = os.path.join(self.experiment_result_save_path, 'best_data_complete.csv')
        metrics_analysis_path = os.path.join(self.experiment_result_save_path, 'metrics_analysis')  
        plt_result_path = os.path.join(self.experiment_result_save_path, 'plt_result.png')
        best_args_path = os.path.join(self.experiment_result_save_path, 'best_args_config.yml')
        final_complete_save_path = os.path.join(self.experiment_result_save_path, 'final_complete.csv')
        create_folder([data_complete_path, metrics_analysis_path])
        
        parser = argparse.ArgumentParser(description='data complete configs')
        parser.add_argument('--data_complete_path', type=str, default=data_complete_path)
        parser.add_argument('--best_data_complete_path', type=str, default=best_data_complete_path)
        parser.add_argument('--metrics_analysis_path', type=str, default=metrics_analysis_path)
        parser.add_argument('--plt_result_path', type=str, default=plt_result_path)
        parser.add_argument('--best_args_path', type=str, default=best_args_path)
        parser.add_argument('--final_complete_save_path', type=str, default=final_complete_save_path)
        args = parser.parse_known_args()[0]
        return args

In [3]:
flow_sim_args = args_config('flow_sim')
flow_zcity_args = args_config('flow_zcity')
pres_sim_args = args_config('pres_sim')
pres_zcity_args = args_config('pres_zcity')

In [4]:
def generate_Psi(dim_time, time_lags):  
    Psis = []
    max_lag = np.max(time_lags)
    for i in range(len(time_lags) + 1):
        row = np.arange(0, dim_time - max_lag)  
        if i == 0:
            col = np.arange(0, dim_time - max_lag) + max_lag
        else:
            col = np.arange(0, dim_time - max_lag) + max_lag - time_lags[i - 1]
        data = np.ones(dim_time - max_lag)
        Psi = sparse.coo_matrix((data, (row, col)), shape = (dim_time - max_lag, dim_time))
        Psis.append(Psi)
    return Psis

In [5]:
def print_result(it, tol, var, var_hat):
    print('Iter: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    num = len(var)
    mape = compute_mape(var, var_hat)
    rmse = compute_rmse(var, var_hat)
    smape = compute_smape(var, var_hat)
    print('complete Num: {}'.format(num))
    print('complete MAPE: {:.6}'.format(mape))
    print('complete RMSE: {:.6}'.format(rmse))
    print('complete SMAPE: {:.6}'.format(smape))
    return mape, rmse, smape

In [6]:
def metrics_analysis(mape_lists, rmse_lists, smape_lists, args, c_search_space):
    mape_df = pd.DataFrame(mape_lists, index=c_search_space, columns=['mape'])
    rmse_df = pd.DataFrame(rmse_lists, index=c_search_space, columns=['rmse'])
    smape_df = pd.DataFrame(smape_lists, index=c_search_space, columns=['smape'])
    mape_df.to_csv(os.path.join(args.metrics_analysis_path, 'mape_all_sensors.csv'), index=True, header=True)
    rmse_df.to_csv(os.path.join(args.metrics_analysis_path, 'rmse_all_sensors.csv'), index=True, header=True)
    smape_df.to_csv(os.path.join(args.metrics_analysis_path, 'smape_all_sensors.csv'), index=True, header=True)
    return

In [7]:
def compute_true_metrics(raw_data_value, denorm_sparse_value, denorm_data_complete_value):
    pos_test = np.where((raw_data_value != 0) & (denorm_sparse_value == 0))
    true_mape = compute_mape(raw_data_value[pos_test], denorm_data_complete_value[pos_test])
    true_rmse = compute_rmse(raw_data_value[pos_test], denorm_data_complete_value[pos_test])
    true_smape = compute_smape(raw_data_value[pos_test], denorm_data_complete_value[pos_test])
    return true_mape, true_rmse, true_smape

In [8]:
def fill_data_after_preprocess(best_data_complete_path, data_after_preprocess_path, data_after_complete_path):
    best_data_complete_df = read_csv_data(best_data_complete_path)
    data_after_preprocess_df = read_csv_data(data_after_preprocess_path)
    data_after_preprocess_df = data_after_preprocess_df.astype('float64')
   
    zero_indices = np.where(data_after_preprocess_df == 0)
    zero_indices = list(zip(zero_indices[0], zero_indices[1]))
    for zero_index in zero_indices:
        data_after_preprocess_df.iloc[zero_index[0], zero_index[1]] = best_data_complete_df.iloc[zero_index[0], zero_index[1]]
    data_after_preprocess_df.to_csv(data_after_complete_path, index=True, header=True)
    print('Data after complete saved in {}'.format(data_after_complete_path))
    return

In [9]:
def svt(mat, tau):
    [m,n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = 0)
        s = np.sqrt(s)
        tol = n * np.finfo(float).eps * np.max(s)
        idx = np.sum(s > max(tau, tol))
        mid = (s[:idx] - tau) / s[:idx]
        return (u[:,:idx] @ np.diag(mid)) @ (u[:,:idx].T @ mat)
    elif m > 2 * n:
        return svt(mat.T, tau).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    return u[:,:idx] @ np.diag(s[:idx]-tau) @ v[:idx,:]

In [10]:
def latc(dense_tensor, sparse_tensor, time_lags, alpha, rho0, lambda0, 
         epsilon = 1e-4, maxiter = 100, K = 3):
    """Low-Rank Autoregressive Tensor Completion (LATC)"""
    
    dim = np.array(sparse_tensor.shape)
    dim_time = int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)  
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))  
    dense_test = dense_tensor[pos_test]
    del dense_tensor
    
    T = np.zeros(dim)
    Z_tensor = sparse_tensor.copy() 
    Z = sparse_mat.copy()
    A = 0.001 * np.random.rand(dim[0], d)
    Psis = generate_Psi(dim_time, time_lags)
    
    iden = sparse.coo_matrix((np.ones(dim_time), (np.arange(0, dim_time), np.arange(0, dim_time))), shape = (dim_time, dim_time))
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    while True:
        temp = []
        for m in range(dim[0]):
            Psis0 = Psis.copy()
            for i in range(d):
                Psis0[i + 1] = A[m, i] * Psis[i + 1]
            B = Psis0[0] - sum(Psis0[1 :])
            temp.append(B.T @ B)
        for k in range(K):
            rho = min(rho * 1.05, 1e5)
            tensor_hat = np.zeros(dim)
            for p in range(len(dim)):
                tensor_hat += alpha[p] * mat2ten(svt(ten2mat(Z_tensor - T / rho, p), alpha[p] / rho), dim, p)
            temp0 = rho / lambda0 * ten2mat(tensor_hat + T / rho, 0)
            mat = np.zeros((dim[0], dim_time))
            for m in range(dim[0]):
                mat[m, :] = spsolve(temp[m] + rho * iden / lambda0, temp0[m, :])
            Z[pos_missing] = mat[pos_missing]
            Z_tensor = mat2ten(Z, dim, 0)
            T = T + rho * (tensor_hat - Z_tensor)
        for m in range(dim[0]):
            A[m, :] = np.linalg.lstsq(Z[m, ind].T, Z[m, max_lag :], rcond = None)[0]

        mat_hat = ten2mat(tensor_hat, 0)
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        if (tol < epsilon) or (it >= maxiter):
            break
    
    tensor_hat[tensor_hat < 1] = 1  
    print_result(it, tol, dense_test, tensor_hat[pos_test]) 
    return tensor_hat

In [11]:
class LATC_dataComplete():

    def __init__(self, dataset_name, experiment_name, c_search_space, experiment_args):
        if dataset_name == 'flow_sim':
            self.raw_dataset_path = flow_sim_path
        elif dataset_name == 'pres_sim':
            self.raw_dataset_path = pres_sim_path
        elif dataset_name == 'flow_zcity':
            self.raw_dataset_path = flow_zcity_path
        elif dataset_name == 'pres_zcity':
            self.raw_dataset_path = pres_zcity_path

        self.raw_data_df = read_csv_data(self.raw_dataset_path)
        self.c_search_space = c_search_space
        self.experiment_args = experiment_args
        
        self.data_norm_for_completing_path = os.path.join(dataset_sparse_path, dataset_name, 'data_norm_for_completing.csv')
        self.sparse_mat_for_completing_path = os.path.join(dataset_sparse_path, dataset_name, experiment_name, 'sparse_mat_for_completing.csv')
        self.denorm_sparse_for_completing_path = os.path.join(dataset_sparse_path, dataset_name, experiment_name, 'denorm_sparse_for_completing.csv')

        self.data_complete_path = self.experiment_args.data_complete_path
        self.best_data_complete_path = self.experiment_args.best_data_complete_path
        self.metrics_analysis_path = self.experiment_args.metrics_analysis_path
        self.plt_result_path = self.experiment_args.plt_result_path
        self.best_args_path = self.experiment_args.best_args_path
        self.final_complete_save_path = self.experiment_args.final_complete_save_path

        self._prepareForImputation()
        return
    
    def _prepareForImputation(self):
        self.norm_data_df = read_csv_data(self.data_norm_for_completing_path)
        self.dense_mat = self.norm_data_df.values
        self.dense_tensor = self.dense_mat.T.reshape([self.dense_mat.shape[1], -1, int(24*60/15)]).transpose(0, 2, 1)

        self.sparse_mat_df = read_csv_data(self.sparse_mat_for_completing_path)
        self.sparse_mat = self.sparse_mat_df.T.values
        self.sparse_tensor = self.sparse_mat.reshape([self.dense_mat.shape[1], -1, int(24*60/15)]).transpose(0, 2, 1)
        self.denorm_sparse_mat_df = read_csv_data(self.denorm_sparse_for_completing_path)
        return

    def latc_complete(self):
       
        mape_all_sensors_lists = []
        rmse_all_sensors_lists = []
        smape_all_sensors_lists = []
        mape_single_sensor_df = pd.DataFrame(columns=self.raw_data_df.columns)
        rmse_single_sensor_df = pd.DataFrame(columns=self.raw_data_df.columns)
        smape_single_sensor_df = pd.DataFrame(columns=self.raw_data_df.columns)
        best_rmse = np.inf  
        best_c = None
        best = False
        current_epoch = 1
        total_epoch = len(self.c_search_space)
        for c in self.c_search_space:
            mape_all_sensors_list = []
            rmse_all_sensors_list = []
            smape_all_sensors_list = []
            start = time.time()
            time_lags = np.arange(1, 7)
            alpha = np.ones(3) / 3
            rho = 1e-5
            lambda0 = c * rho
            print('epoch: %d/%d, c: %.2f' % (current_epoch, total_epoch, c))
            self.tensor_hat = latc(self.dense_tensor, self.sparse_tensor, time_lags, alpha, rho, lambda0)
            self.mat_hat = ten2mat(self.tensor_hat, 0)
            self.data_complete_df = pd.DataFrame(self.mat_hat.T, index=self.norm_data_df.index, columns=self.norm_data_df.columns)
            self.denorm_data_complete_df = reverse_one2hundred_normalization(self.data_complete_df, self.raw_data_df)
            save_path = os.path.join(self.data_complete_path, 'c=' + str(c) + '.csv')
            
            self.denorm_data_complete_df = self.denorm_data_complete_df.round(3)
            self.denorm_data_complete_df.to_csv(save_path, index=True, header=True)

            mape_all_sensors, rmse_all_sensors, smape_all_sensors = compute_true_metrics(self.raw_data_df.values, self.denorm_sparse_mat_df.values, self.denorm_data_complete_df.values)  
            print('MAPE: %.6f, RMSE: %.6f, SMAPE: %.6f' % (mape_all_sensors, rmse_all_sensors, smape_all_sensors))
            if rmse_all_sensors < best_rmse:
                best_rmse = rmse_all_sensors
                best_c = c  
                best = True
            mape_all_sensors_list.append(mape_all_sensors)
            rmse_all_sensors_list.append(rmse_all_sensors)
            smape_all_sensors_list.append(smape_all_sensors)

            if best:
                self.denorm_data_complete_df.to_csv(self.best_data_complete_path, index=True, header=True)
                best = False
            mape_single_sensor_list, rmse_single_sensor_list, smape_single_sensor_list = result_analysis(self.raw_data_df, self.denorm_sparse_mat_df, self.denorm_data_complete_df)
            mape_single_sensor_df.loc['c=' + str(c), :] = mape_single_sensor_list
            rmse_single_sensor_df.loc['c=' + str(c), :] = rmse_single_sensor_list
            smape_single_sensor_df.loc['c=' + str(c), :] = smape_single_sensor_list
            end = time.time()
            print('Running time: %d seconds\n' %(end - start))
            current_epoch += 1
            mape_all_sensors_lists.append(mape_all_sensors_list)
            rmse_all_sensors_lists.append(rmse_all_sensors_list)
            smape_all_sensors_lists.append(smape_all_sensors_list)
        mape_all_sensors_lists = np.array(mape_all_sensors_lists)
        rmse_all_sensors_lists = np.array(rmse_all_sensors_lists)
        smape_all_sensors_lists = np.array(smape_all_sensors_lists)
        
        metrics_analysis(mape_all_sensors_lists, rmse_all_sensors_lists, smape_all_sensors_lists, self.experiment_args, self.c_search_space)
        mape_single_sensor_df.to_csv(os.path.join(self.metrics_analysis_path, 'mape_single_sensor.csv'), index=True, header=True)
        rmse_single_sensor_df.to_csv(os.path.join(self.metrics_analysis_path, 'rmse_single_sensor.csv'), index=True, header=True)
        smape_single_sensor_df.to_csv(os.path.join(self.metrics_analysis_path, 'smape_single_sensor.csv'), index=True, header=True)
        
        best_mape_single_sensor_list = mape_single_sensor_df.loc['c=' + str(best_c), :].values
        best_rmse_single_sensor_list = rmse_single_sensor_df.loc['c=' + str(best_c), :].values
        best_smape_single_sensor_list = smape_single_sensor_df.loc['c=' + str(best_c), :].values
        return best_c, best_mape_single_sensor_list, best_rmse_single_sensor_list, best_smape_single_sensor_list
    
    def run(self):
        print('\n--------------------------- Beginning ---------------------------')
        best_c, best_mape_single_sensor_list, best_rmse_single_sensor_list, best_smape_single_sensor_list = self.latc_complete()
        
        self.best_args_dict = {}
        self.best_args_dict['best_c'] = best_c
        write_yaml(self.best_args_path, self.best_args_dict)
        fill_data_after_preprocess(self.best_data_complete_path, self.raw_dataset_path, self.final_complete_save_path)
        plt_result(self.raw_data_df, read_csv_data(self.best_data_complete_path), self.denorm_sparse_mat_df, self.plt_result_path,
                   best_mape_single_sensor_list, best_rmse_single_sensor_list, best_smape_single_sensor_list)
        print('\n--------------------------- Ending ---------------------------')         


# 2 Experiments

------------

## 2.1 30% Random Missing

In [12]:
missing_type = 'random'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

## 2.2 60% Random Missing

In [ ]:
missing_type = 'random'
missing_rate = 0.6
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

------------

## 2.3 30% Long-Range Missing

In [ ]:
missing_type = 'long_range'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

-----

## 2.4 30% Block Missing

In [ ]:
missing_type = 'block'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

------

## 2.5 30% Mix Missing

In [ ]:
missing_type = 'mix'
missing_rate = 0.3
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

## 2.6 50% Mix Missing

In [ ]:
missing_type = 'mix'
missing_rate = 0.5
experiment_name = missing_type + '_' + str(missing_rate)

### flow_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_sim'

experiment_args = flow_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_sim

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_sim'

experiment_args = pres_sim_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### flow_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'flow_zcity'

experiment_args = flow_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()

### pres_zcity

In [ ]:
c_search_space = [1/10, 1/5, 1, 5, 10]
dataset_name = 'pres_zcity'

experiment_args = pres_zcity_args.experiment_config(experiment_name)
Imputer = LATC_dataComplete(dataset_name, experiment_name, c_search_space, experiment_args)
Imputer.run()